# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
weather = pd.read_csv('../output_data/city_data.csv')
weather.head()

City  Cloudiness Country        Date  Humidity    Lat     Lng  \
0            kodiak           1      US  1584132989        59  57.79 -152.41   
1       yellowknife           5      CA  1584132816        57  62.46 -114.35   
2             sitka          20      US  1584132752        26  57.05 -135.33   
3  south lake tahoe           1      US  1584133602        16  38.93 -119.98   
4            tiarei          20      PF  1584133440        66 -17.53 -149.33   

   Max Temp  Wind Speed  Total  
0      32.0        3.36   0.74  
1      -9.0        4.76   5.87  
2      33.8        4.70   6.22  
3      62.6       12.75  11.30  
4      87.8        4.70  11.64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
gmaps.configure(api_key= g_key)
locations = weather[["Lat", "Lng"]]
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=weather["Humidity"],dissipating=False, max_intensity=100,point_radius = 4)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
#weather= weather.dropna()

weather_1 = weather[(weather["Max Temp"] > 70) & 
                        (weather["Max Temp"] < 80) &  
                        (weather["Wind Speed"] < 10) &
                        (weather["Cloudiness"] == 0)
                       ]
weather_1.head(5)

City  Cloudiness Country        Date  Humidity    Lat    Lng  \
35             castro           0      BR  1584132614        47 -24.79 -50.01   
64           cidreira           0      BR  1584132788        87 -30.18 -50.21   
67               atar           0      MR  1584133415        10  20.52 -13.05   
70              adrar           0      MR  1584133496         9  20.50 -10.07   
80  sao joao da ponte           0      BR  1584133622        89 -15.93 -44.01   

    Max Temp  Wind Speed   Total  
35     75.83        3.60   51.63  
64     77.27        8.01   91.89  
67     71.15        5.14   93.76  
70     71.51        5.64   96.58  
80     73.67        5.03  107.76

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
weather_1['Hotel'] = ""
weather_1['Hotel lat'] = ""
weather_1['Hotel lng'] = ""

weather_1.head()

C:\Users\Prajakta\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Prajakta\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Prajakta\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

City  Cloudiness Country        Date  Humidity    Lat    Lng  \
35             castro           0      BR  1584132614        47 -24.79 -50.01   
64           cidreira           0      BR  1584132788        87 -30.18 -50.21   
67               atar           0      MR  1584133415        10  20.52 -13.05   
70              adrar           0      MR  1584133496         9  20.50 -10.07   
80  sao joao da ponte           0      BR  1584133622        89 -15.93 -44.01   

    Max Temp  Wind Speed   Total Hotel Hotel lat Hotel lng  
35     75.83        3.60   51.63                            
64     77.27        8.01   91.89                            
67     71.15        5.14   93.76                            
70     71.51        5.64   96.58                            
80     73.67        5.03  107.76

In [26]:
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in weather_1.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    
    try:
        weather_1.loc[index, "Hotel"] = name_address["results"][0]["name"]
        weather_1.loc[index, "Hotel lat"] = name_address["results"][0]["geometry"]["location"]["lat"]
        weather_1.loc[index, "Hotel lng"] = name_address["results"][0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
weather_1.head()

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


City  Cloudiness Country        Date  Humidity    Lat    Lng  \
35             castro           0      BR  1584132614        47 -24.79 -50.01   
64           cidreira           0      BR  1584132788        87 -30.18 -50.21   
67               atar           0      MR  1584133415        10  20.52 -13.05   
70              adrar           0      MR  1584133496         9  20.50 -10.07   
80  sao joao da ponte           0      BR  1584133622        89 -15.93 -44.01   

    Max Temp  Wind Speed   Total                             Hotel Hotel lat  \
35     75.83        3.60   51.63  Barbur Center Hotel Ponta Grossa  -25.0953   
64     77.27        8.01   91.89                    Hotel Ecovilly  -29.8305   
67     71.15        5.14   93.76                      Odar kanawal   20.4909   
70     71.51        5.64   96.58                                               
80     73.67        5.03  107.76                   SUN ROCKS HOTEL  -16.2117   

   Hotel lng  
35  -50.1642  
64  -50.5332  
67   -13.063  
70            
80  -44.4334

In [29]:
hotel_df = weather_1[["Hotel", "Hotel lat", "Hotel lng"]]
hotel_df.head(5)

Hotel Hotel lat Hotel lng
35  Barbur Center Hotel Ponta Grossa  -25.0953  -50.1642
64                    Hotel Ecovilly  -29.8305  -50.5332
67                      Odar kanawal   20.4909   -13.063
70                                                      
80                   SUN ROCKS HOTEL  -16.2117  -44.4334

In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in weather_1.iterrows()]
locations = hotel_df[["Hotel lat", "Hotel lng"]]

In [34]:
# Display Map
locations = weather_1[["Lat","Lng"]].astype(float)

figure_layout = {
    'width': '600px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

markers = gmaps.marker_layer(
    locations, info_box_content= hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…